In [1]:
import psycopg2
from config import load_config
import pandas as pd
import folium
import os
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
import numpy as np

In [13]:
def capturar_data_mapa(conn, table_name, linha):
    cur = conn.cursor()
    all_data = [] 
    try:
        fetch_query = f'''
        SELECT latitude::double precision, longitude::double precision
        FROM {table_name}
        WHERE linha='{linha}'
        '''
        cur.execute(fetch_query)
        rows = cur.fetchall()
        all_data.extend(rows)
    except Exception as e:
            print(f"Erro ao executar a query na tabela {table_name}: {e}")
            conn.rollback()
    else:
         conn.commit()
    cur.close()
    return all_data

In [14]:
def limpar_dados_tabela(table_name, conn):
    cur = conn.cursor()
    cur.execute(f"""
        ALTER TABLE {table_name} ADD COLUMN IF NOT EXISTS hour INTEGER;
        UPDATE {table_name} SET hour = EXTRACT(HOUR FROM TO_TIMESTAMP(datahora / 1000));
        
        CREATE TABLE {table_name}_filter AS
        SELECT *
        FROM {table_name}
        WHERE hour BETWEEN 8 AND 22;
        
        ALTER TABLE {table_name} DROP COLUMN hour;
        ALTER TABLE {table_name}_filter DROP COLUMN hour;
    """)
    cur.close()
    conn.commit()

In [15]:
def adicionar_geom(table_name, conn):
    cur = conn.cursor()
    try:
        cur.execute(f"""
            ALTER TABLE {table_name}_filter ADD COLUMN IF NOT EXISTS geom geography(Point, 4326);
        """)
        cur.execute(f"""
            UPDATE {table_name}_filter SET geom = ST_SetSRID(ST_MakePoint(longitude::double precision, latitude::double precision), 4326);
        """)
        print(f"Coluna geom criada e preenchida na tabela {table_name}_filter.")
    except Exception as e:
        print(f"Erro ao atualizar a tabela {table_name}_filter: {e}")
        conn.rollback()
    else:
         conn.commit()
    cur.close()

In [16]:
def filtra_linhas(table_name, linhas_de_interesse, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_temp'
    linhas_str = ', '.join([f"'{linha}'" for linha in linhas_de_interesse])
    try:
        # Criar tabela temporária filtrada
        cur.execute(f"""
            CREATE TEMP TABLE {temp_table_name} AS
            SELECT *
            FROM {table_name}_filter
            WHERE linha IN ({linhas_str});
        """)
        print(f"Tabela temporária {temp_table_name} criada com sucesso.")
        conn.commit()
    except Exception as e:
        print(f"Erro ao criar a tabela temporária {temp_table_name}: {e}")
        conn.rollback()
    cur.close()

In [17]:
def sobrescreve_tabelas(table_name, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_temp'
    try:
        # Excluir a tabela original
        cur.execute(f"DROP TABLE {table_name}_filter;")
        # Renomear a tabela temporária para o nome original
        cur.execute(f"ALTER TABLE {temp_table_name} RENAME TO {table_name}_filter;")
        print(f"Tabela {table_name}_filter sobrescrita com sucesso.")
        conn.commit()
    except Exception as e:
        print(f"Erro ao sobrescrever a tabela {table_name}: {e}")
        conn.rollback()
    cur.close()

In [7]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [8]:
config = load_config()
conn = connect(config)

Connected to the PostgreSQL server.


In [9]:
linhas_de_interesse = ['483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', '852', '557', '759', '343', '779', '905', '108']

In [10]:
table_names = ['dia_0105', 'dia_0205', 'dia_0305', 'dia_0405', 'dia_0505', 'dia_0605', 'dia_0705', 'dia_0805', 'dia_0905', 'dia_1005', 'dia_2504', 'dia_2604', 'dia_2704', 'dia_2804', 'dia_2904', 'dia_3004']

In [11]:
for table in table_names:
    limpar_dados_tabela(table, conn)

KeyboardInterrupt: 

In [9]:
for table in table_names:
    adicionar_geom(table, conn)

Coluna geom criada e preenchida na tabela dia_0105_filter.
Coluna geom criada e preenchida na tabela dia_0205_filter.
Coluna geom criada e preenchida na tabela dia_0305_filter.
Coluna geom criada e preenchida na tabela dia_0405_filter.
Coluna geom criada e preenchida na tabela dia_0505_filter.
Coluna geom criada e preenchida na tabela dia_0605_filter.
Coluna geom criada e preenchida na tabela dia_0705_filter.
Coluna geom criada e preenchida na tabela dia_0805_filter.
Coluna geom criada e preenchida na tabela dia_0905_filter.
Coluna geom criada e preenchida na tabela dia_1005_filter.
Coluna geom criada e preenchida na tabela dia_2504_filter.
Coluna geom criada e preenchida na tabela dia_2604_filter.
Coluna geom criada e preenchida na tabela dia_2704_filter.
Coluna geom criada e preenchida na tabela dia_2804_filter.
Coluna geom criada e preenchida na tabela dia_2904_filter.
Coluna geom criada e preenchida na tabela dia_3004_filter.


In [57]:
"""for table in table_names:
    filtra_linhas(table, linhas_de_interesse, conn)
    sobrescreve_tabelas(table, conn)"""

Tabela temporária dia_0105_temp criada com sucesso.
Tabela dia_0105_filter sobrescrita com sucesso.
Tabela temporária dia_0205_temp criada com sucesso.
Tabela dia_0205_filter sobrescrita com sucesso.
Tabela temporária dia_0305_temp criada com sucesso.
Tabela dia_0305_filter sobrescrita com sucesso.
Tabela temporária dia_0405_temp criada com sucesso.
Tabela dia_0405_filter sobrescrita com sucesso.
Tabela temporária dia_0505_temp criada com sucesso.
Tabela dia_0505_filter sobrescrita com sucesso.
Tabela temporária dia_0605_temp criada com sucesso.
Tabela dia_0605_filter sobrescrita com sucesso.
Tabela temporária dia_0705_temp criada com sucesso.
Tabela dia_0705_filter sobrescrita com sucesso.
Tabela temporária dia_0805_temp criada com sucesso.
Tabela dia_0805_filter sobrescrita com sucesso.
Tabela temporária dia_0905_temp criada com sucesso.
Tabela dia_0905_filter sobrescrita com sucesso.
Tabela temporária dia_1005_temp criada com sucesso.
Tabela dia_1005_filter sobrescrita com sucesso.


In [39]:
"""for linha in linhas_de_interesse:
    data = capturar_data_mapa(conn, 'dia_2704_filter', linha)
    if data:
        avg_latitude = sum([d[0] for d in data]) / len(data)
        avg_longitude = sum([d[1] for d in data]) / len(data)
        m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)
        # Adicione os pontos ao mapa
        for lat, lon in data:
            folium.CircleMarker(location=[lat, lon], radius=1, color='blue').add_to(m)
        output_path = os.path.join(f'trajetos/output_map_{linha}.html')
        m.save(output_path)
        print(f"Mapa salvo em {output_path}")"""

Mapa salvo em trajetos/output_map_483.html
Mapa salvo em trajetos/output_map_864.html
Mapa salvo em trajetos/output_map_639.html
Mapa salvo em trajetos/output_map_3.html
Mapa salvo em trajetos/output_map_309.html
Mapa salvo em trajetos/output_map_774.html
Mapa salvo em trajetos/output_map_629.html
Mapa salvo em trajetos/output_map_371.html
Mapa salvo em trajetos/output_map_397.html
Mapa salvo em trajetos/output_map_100.html
Mapa salvo em trajetos/output_map_838.html
Mapa salvo em trajetos/output_map_315.html
Mapa salvo em trajetos/output_map_624.html
Mapa salvo em trajetos/output_map_388.html
Mapa salvo em trajetos/output_map_918.html
Mapa salvo em trajetos/output_map_665.html
Mapa salvo em trajetos/output_map_328.html
Mapa salvo em trajetos/output_map_497.html
Mapa salvo em trajetos/output_map_878.html
Mapa salvo em trajetos/output_map_355.html
Mapa salvo em trajetos/output_map_606.html
Mapa salvo em trajetos/output_map_457.html
Mapa salvo em trajetos/output_map_550.html
Mapa salvo em

In [10]:
dados = pd.read_sql("SELECT * FROM dia_2704_filter WHERE linha='497'", conn)

/tmp/ipykernel_25759/2021440370.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dados = pd.read_sql("SELECT * FROM dia_2704_filter WHERE linha='497'", conn)


In [11]:
dados.loc[dados["ordem"] == 'B31125'].head(100)

,id,ordem,latitude,longitude,datahora,velocidade,linha,datahoraenvio,datahoraservidor,geom
16,430,B31125,-22.80473,-43.30919,1714229983000,0,497,1714230005000,1714230010000,0101000020E61000000820B58993A745C02E1C08C902CE...
44,18103,B31125,-22.80473,-43.30919,1714230163000,0,497,1714230178000,1714230193000,0101000020E61000000820B58993A745C02E1C08C902CE...
60,35743,B31125,-22.80473,-43.30919,1714230343000,0,497,1714230352000,1714230377000,0101000020E61000000820B58993A745C02E1C08C902CE...
81,38874,B31125,-22.80473,-43.30919,1714230373000,0,497,1714230383000,1714230407000,0101000020E61000000820B58993A745C02E1C08C902CE...
104,42045,B31125,-22.80473,-43.30919,1714230403000,0,497,1714230414000,1714230438000,0101000020E61000000820B58993A745C02E1C08C902CE...
...,...,...,...,...,...,...,...,...,...,...
1762,266020,B31125,-22.80478,-43.30928,1714232624000,0,497,1714232632000,1714232642000,0101000020E610000048A7AE7C96A745C09279E40F06CE...
1801,281486,B31125,-22.80478,-43.30928,1714232773000,0,497,1714232786000,1714232795000,0101000020E610000048A7AE7C96A745C09279E40F06CE...
1809,269089,B31125,-22.80478,-43.30928,1714232653000,0,497,1714232662000,1714232672000,0101000020E610000048A7AE7C96A745C09279E40F06CE...
1825,275160,B31125,-22.80478,-43.30928,1714232713000,0,497,1714232724000,1714232734000,0101000020E610000048A7AE7C96A745C09279E40F06CE...


In [67]:
query = """
WITH ordered_points AS (
    SELECT
        ordem,
        linha,
        geom,
        TO_TIMESTAMP(datahora / 1000) AS datahora_ts,
        LAG(TO_TIMESTAMP(datahora / 1000)) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_datahora_ts,
        LAG(geom) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_geom,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS rn
    FROM dia_2704_filter
),
same_position_periods AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        EXTRACT(EPOCH FROM (datahora_ts - prev_datahora_ts)) AS duration,
        CASE 
            WHEN ST_DWithin(geom, prev_geom, 100) THEN 1
            ELSE 0
        END AS is_stationary,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY datahora_ts) - 
        ROW_NUMBER() OVER (PARTITION BY ordem, CASE WHEN ST_DWithin(geom, prev_geom, 200) THEN 1 ELSE 0 END ORDER BY datahora_ts) AS grp
    FROM ordered_points
),
stationary_groups AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        duration,
        is_stationary,
        grp
    FROM same_position_periods
    WHERE is_stationary = 1
)
SELECT
    ordem,
    linha,
    MIN(prev_datahora_ts) AS start_time,
    MAX(datahora_ts) AS end_time,
    SUM(duration) AS total_duration,
    ST_X(geom::geometry) AS longitude,
    ST_Y(geom::geometry) AS latitude,
    grp
FROM stationary_groups
GROUP BY ordem, linha, grp, geom
HAVING SUM(duration) >= 600; -- 20 minutes in seconds
"""

In [68]:
df = pd.read_sql(query, conn)

/tmp/ipykernel_25759/1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


       ordem linha                start_time                  end_time  \
2050  B31005   483 2024-04-27 11:42:28+00:00 2024-04-27 12:09:20+00:00   
2051  B31005   483 2024-04-27 12:28:25+00:00 2024-04-27 18:33:41+00:00   
2052  B31005   483 2024-04-27 19:38:48+00:00 2024-04-27 19:53:54+00:00   
2053  B31005   483 2024-04-27 19:08:19+00:00 2024-04-27 19:38:48+00:00   
2054  B31005   483 2024-04-27 20:10:21+00:00 2024-04-28 01:12:34+00:00   
...      ...   ...                       ...                       ...   
2354  B31167   483 2024-04-27 22:15:32+00:00 2024-04-27 22:45:32+00:00   
2355  B31167   483 2024-04-27 22:45:32+00:00 2024-04-27 23:15:32+00:00   
2356  B31167   483 2024-04-27 23:15:32+00:00 2024-04-27 23:45:32+00:00   
2357  B31167   483 2024-04-27 13:47:18+00:00 2024-04-27 15:31:09+00:00   
2358  B31167   483 2024-04-27 11:25:03+00:00 2024-04-27 11:55:03+00:00   

      total_duration  longitude  latitude  grp  
2050          1612.0  -43.31036 -22.80417    1  
2051         